Test Cases covered in this .ipynb file

16. CNN Pretrain ternary

Remaining will be covered in different .ipynb due to memory constraints

## CNN pretrained ternary 

In [1]:
### import all the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gensim
from gensim import corpora, similarities, models
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from gensim.models import KeyedVectors
import gensim.downloader as api
import gc

In [2]:
### check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU")

No GPU available, using CPU


In [3]:
### read the tsv file
amazon_df = pd.read_csv('https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz',sep = '\t', on_bad_lines='skip')
amazon_df.dropna(inplace=True)
amazon_df.head(10)

/var/folders/z2/ns28nw4n5sv4_r39d2j5pw_80000gp/T/ipykernel_6830/3191911374.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_df = pd.read_csv('https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz',sep = '\t', on_bad_lines='skip')


marketplace  customer_id       review_id  product_id  product_parent  \
0          US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1          US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2          US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3          US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4          US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   
5          US     21751234   R8T6MO75ND212  B004J2NBCO       214932869   
6          US      9109358  R2YWMQT2V11XYZ  B00MOPAG8K       863351797   
7          US      9967215  R1V2HYL6OI9V39  B003AHIK7U       383470576   
8          US     11234247  R3BLQBKUNXGFS4  B006TKH2RO       999128878   
9          US     12731488  R17MOWJCAR9Y8Q  B00W61M9K0       622066861   

                                       product_title product_category  \
0     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet  Office Products   
1          Dust-Off Compressed Gas Duster, Pack of 4  Office Products   
2  Amram Tagger Standard Tag Attaching Tagging Gu...  Office Products   
3  AmazonBasics 12-Sheet High-Security Micro-Cut ...  Office Products   
4  Derwent Colored Pencils, Inktense Ink Pencils,...  Office Products   
5  Quartet Magnetic Dry-Erase Weekly Organizer, 6...  Office Products   
6  KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...  Office Products   
7              Bible Dry Highlighting Kit (Set of 4)  Office Products   
8  Parker Ingenuity Large Black Rubber & Metal CT...  Office Products   
9                                RFID Card Protector  Office Products   

  star_rating  helpful_votes  total_votes vine verified_purchase  \
0           5            0.0          0.0    N                 Y   
1           5            0.0          1.0    N                 Y   
2           5            0.0          0.0    N                 Y   
3           1            2.0          3.0    N                 Y   
4           4            0.0          0.0    N                 Y   
5           5            0.0          0.0    N                 Y   
6           5            0.0          0.0    N                 N   
7           5            6.0          6.0    N                 Y   
8           5            0.0          0.0    N                 Y   
9           5            0.0          0.0    N                 Y   

                                     review_headline  \
0                                         Five Stars   
1  Phffffffft, Phfffffft. Lots of air, and it's C...   
2                      but I am sure I will like it.   
3  and the shredder was dirty and the bin was par...   
4                                         Four Stars   
5                                         Five Stars   
6                                         Five Stars   
7                         Won't bleed through pages!   
8                                          Nice pen!   
9                                Credit card sleeves   

                                         review_body review_date  
0                                     Great product.  2015-08-31  
1  What's to say about this commodity item except...  2015-08-31  
2    Haven't used yet, but I am sure I will like it.  2015-08-31  
3  Although this was labeled as &#34;new&#34; the...  2015-08-31  
4                    Gorgeous colors and easy to use  2015-08-31  
5  Perfect for planning weekly meals. Removrd the...  2015-08-31  
6  Gold plated fusers are the best! It will never...  2015-08-31  
7  I have used these highlighters for my bible fo...  2015-08-31  
8  Heavy pen that writes very well.  I've had it ...  2015-08-31  
9  Not sure if they work but sent quickly and fit...  2015-08-31

In [4]:
### add class label 1  2 3
def label_class(rating):
    if int(rating)>=4:
        return 0
    elif int(rating)<3:
        return 1
    else:
        return 2

amazon_df['Ratings']=amazon_df['star_rating'].apply(label_class)
amazon=amazon_df.copy()
amazon_df1=amazon.query(" Ratings ==0 ").sample(n=50000, replace=True)
amazon_df2 = amazon.query(" Ratings ==1 ").sample(n=50000, replace=True)
amazon_df3 = amazon.query(" Ratings ==2 ").sample(n=50000, replace=True)

amazon_df_final3=pd.concat([amazon_df1, amazon_df2, amazon_df3], axis=0)
amazon_df_final=amazon_df_final.sample(frac = 1)
amazon_df_final3=amazon_df_final3.sample(frac = 1)

In [5]:
X_train3,X_test3,Y_train3,Y_test3=train_test_split(amazon_df_final3['review_body'],amazon_df_final3['Ratings'],test_size=0.2)
print(X_train3.shape,Y_train3.shape)
print(X_test3.shape,Y_test3.shape)

(120000,) (120000,)
(30000,) (30000,)


In [ ]:
del amazon_df_final3, amazon_df1, amazon_df2, amazon_df3, amazon, amazon_df
gc.collect()

In [6]:
model = api.load("word2vec-google-news-300")
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/payalrashinkar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

========================================

In [8]:
print(X_train3.shape,Y_train3.shape)
print(X_test3.shape,Y_test3.shape)

(120000,) (120000,)
(30000,) (30000,)


In [10]:
#### convert words to vectors 
corp = X_train3.values.tolist()
tok_corp = [nltk.word_tokenize(sent) for sent in corp]

word2vec_vectors = []
max_length = 50  
embedding_dim = 300 
batch_size = 100

for sentence in tok_corp:    
    sentence = sentence[:max_length] if len(sentence) > max_length else sentence + [None] * (max_length - len(sentence))    
    sentence_vector = np.zeros((max_length, embedding_dim))    
    for i, word in enumerate(sentence):
        if word in model:
            sentence_vector[i] = model[word]    
    sentence_vector = sentence_vector.reshape(-1)
    word2vec_vectors.append(sentence_vector)
X_train3_word2vec = np.array(word2vec_vectors)

### Convert word2vec into tensor

X_train3_word2vec_cnn = torch.Tensor(X_train3_word2vec).to(device)
Y_train3_cnn = torch.Tensor(Y_train3.to_numpy()).to(device)
Y_train3_cnn = (Y_train3_cnn == 2).float()
dataset3 = TensorDataset(X_train3_word2vec_cnn.to(device), Y_train3_cnn)

### train3_loader
train3_loader_te_1 = DataLoader(dataset3, batch_size=batch_size, shuffle=True)

In [11]:
%%time
del X_train3_word2vec, X_train3, word2vec_vectors, sentence_vector, sentence, corp, tok_corp
gc.collect()

CPU times: user 792 ms, sys: 5.4 s, total: 6.2 s
Wall time: 1min 18s


0

In [12]:
%%time
corp = X_test3.values.tolist()
tok_corp = [nltk.word_tokenize(sent) for sent in corp]
word2vec_vectors = []

for sentence in tok_corp:
    sentence = sentence[:max_length] if len(sentence) > max_length else sentence + [None] * (max_length - len(sentence))    
    sentence_vector = np.zeros((max_length, embedding_dim))
        
    for i, word in enumerate(sentence):
        if word in model:
            sentence_vector[i] = model[word]
    sentence_vector = sentence_vector.reshape(-1)
    word2vec_vectors.append(sentence_vector)

X_test3_word2vec = np.array(word2vec_vectors)

### Convert word2vec into tensor
X_test3_wordtovec_cnn = torch.Tensor(X_test3_word2vec).to(device)
Y_test3_cnn = torch.tensor(Y_test3.to_numpy(), dtype=torch.long).to(device)
dataset_test3 = TensorDataset(X_test3_wordtovec_cnn, Y_test3_cnn)

### test3 loader
test3_loader_te_1 = DataLoader(dataset_test3, batch_size=batch_size, shuffle=True)

CPU times: user 16.6 s, sys: 11.4 s, total: 27.9 s
Wall time: 42.5 s


In [ ]:
class SentimentCNN(nn.Module):
    def __init__(self, max_review_length, embedding_dim, num_classes=2):
        super(SentimentCNN, self).__init__()
        self.embedding_dim = embedding_dim
        self.max_review_length = max_review_length
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=50, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=5, padding=2)
        #self.fc = nn.Linear(10 * max_review_length, num_classes)  # Adjusted for the reshaping
        self.fc = nn.Linear(10, num_classes)

    def forward(self, x):
        # Reshape x to (batch_size, embedding_dim, max_review_length)
        x = x.view(-1, self.embedding_dim, self.max_review_length)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.avg_pool1d(x, x.size(2))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
def train(model, train_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
        
            # Ensure inputs and labels are on the right device (e.g., GPU if you're using one)
            inputs = inputs.to(device)  # Add this if using a GPU
            labels = labels.to(device).long()  # Convert labels to long and send to GPU if needed

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [16]:
ternary_model = SentimentCNN(max_review_length=50, embedding_dim=300, num_classes=3)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer3 = torch.optim.Adam(ternary_model.parameters())

In [18]:
%%time
train(ternary_model, train3_loader_te_1, criterion, optimizer3)

Epoch 1, Loss: 0.6371253728866577
Epoch 2, Loss: 0.6055887937545776
Epoch 3, Loss: 0.56015944480896
Epoch 4, Loss: 0.6131231784820557
Epoch 5, Loss: 0.5197523236274719
CPU times: user 3min 24s, sys: 1min 31s, total: 4min 55s
Wall time: 25min 59s


In [25]:
%%time
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item() * 1.5
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.39 µs


In [26]:
evaluate(ternary_model, test3_loader_te_1)

Accuracy on the test set: 53.19%
